In [2]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor

# 노트북 안에 그래프를 그리기 위해
%matplotlib inline

# 그래프에서 격자로 숫자 범위가 눈에 잘 띄도록 ggplot 스타일을 사용
plt.style.use('ggplot')

import warnings
warnings.filterwarnings('ignore')

In [22]:
train

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,hour,dayofweek
3860,2011-09-11 20:00:00,3,0,0,1,25.42,29.545,73,7.0015,65,161,226,2011,9,20,6
4262,2011-10-09 15:00:00,4,0,0,1,27.88,31.820,51,11.0014,198,209,407,2011,10,15,6
8586,2012-07-19 03:00:00,3,0,1,1,29.52,34.090,70,7.0015,1,4,5,2012,7,3,3
5268,2011-12-13 14:00:00,4,0,1,1,16.40,20.455,37,16.9979,18,116,134,2011,12,14,1
9870,2012-10-15 15:00:00,4,0,1,1,24.60,28.790,78,22.0028,70,190,260,2012,10,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7902,2012-06-09 15:00:00,2,0,0,1,34.44,35.605,20,19.9995,218,337,555,2012,6,15,5
3313,2011-08-07 23:00:00,3,0,0,1,28.70,33.335,79,7.0015,12,43,55,2011,8,23,6
7890,2012-06-09 03:00:00,2,0,0,1,25.42,31.060,57,0.0000,12,28,40,2012,6,3,5
4995,2011-12-02 05:00:00,4,0,1,1,9.02,12.880,80,6.0032,0,23,23,2011,12,5,4


In [3]:
train = pd.read_csv("data/train.csv", parse_dates=["datetime"])
train.shape

(10886, 12)

In [4]:
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['hour'] = train['datetime'].dt.hour
train['dayofweek'] = train['datetime'].dt.dayofweek
train.shape

(10886, 16)

In [5]:
from sklearn.utils import shuffle
train = shuffle(train)

train_70 = train[:int(len(train)*0.7)]
train_30 = train[int(len(train)*0.7):]

In [24]:
train_70.shape

(7620, 16)

In [25]:
train_30.shape

(3266, 16)

In [6]:
f_columns = ['season','holiday','workingday','weather','temp','atemp','casual','registered','year','month','hour','dayofweek']

X_train = train_70[f_columns]
Y_train = train_70['count']

X_test = train_30[f_columns]
Y_test = train_30['count']

# Score
## RMSLE

오차(Error)를 제곱(Square)해서 평균(Mean)한 값의 제곱근(Root) 으로 값이 작을 수록 정밀도가 높다. 

0에 가까운 값이 나올 수록 정밀도가 높은 값이다.

Submissions are evaluated one the Root Mean Squared Logarithmic Error (RMSLE)

$$ \sqrt{\frac{1}{n} \sum_{i=1}^n (\log(p_i + 1) - \log(a_i+1))^2 } $$

In [7]:
from sklearn.metrics import make_scorer

def rmsle(predicted_values, actual_values):
    # 넘파이로 배열 형태로 바꿔준다.
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)
    
    # 예측값과 실제 값에 1을 더하고 로그를 씌워준다.
    log_predict = np.log(predicted_values + 1)
    log_actual = np.log(actual_values + 1)
    
    # 위에서 계산한 예측값에서 실제값을 빼주고 제곱을 해준다.
    difference = log_predict - log_actual
    # difference = (log_predict - log_actual) ** 2
    difference = np.square(difference)
    
    # 평균을 낸다.
    mean_difference = difference.mean()
    
    # 다시 루트를 씌운다.
    score = np.sqrt(mean_difference)
    
    return score

rmsle_scorer = make_scorer(rmsle)
rmsle_scorer

make_scorer(rmsle)

## 앙상블모델 - 배깅
### Ensemble Model - Bagging

* bootstrap aggregating의 줄임말
* 안정성과 정확도를 향상시키기 위해 고안된 일종의 앙상블 학습법의 메타 알고리즘
* 분산을 줄이고 과적합(overfitting)을 피하도록 해준다

In [8]:
import time
from sklearn.ensemble import BaggingRegressor

start = time.time()

model = BaggingRegressor()

model.fit(X_train,Y_train)

time_len = (time.time() - start)

print('정확도 : {}'.format(rmsle(model.predict(X_test), Y_test)))
print('수행시간 : {}'.format(time_len))

정확도 : 0.016714247533544172
수행시간 : 0.27152204513549805


## 랜덤포레스트
### Ensemble Model - Random Forest
* 과적합을 피하고자 다수의 결정트리를 사용
* 월등히 높은 정확성
* 간편하고 빠른 학습 및 테스트 알고리즘
* 변수소거 없이 수천 개의 입력 변수들을 다루는 것이 가능
* 임의화를 통한 좋은 일반화 성능
* 다중 클래스 알고리즘 특성

In [9]:
from sklearn.ensemble import RandomForestRegressor

start = time.time()

model = RandomForestRegressor()

model.fit(X_train,Y_train) 

time_len = (time.time() - start)

print('정확도 : {}'.format(rmsle(model.predict(X_test), Y_test)))
print('수행시간 : {}'.format(time_len))

정확도 : 0.017258102948584254
수행시간 : 0.24701929092407227


## 에이다부스트
###  AdaBoost
* 이전의 분류기에 의해 잘못 분류된 것들을 이어지는 약한 학습기들이 수정해줄 수 있다는 점에서 다양한 상황에 적용할 수 있다
* 다른 학습 알고리즘보다 과적합(overfitting)에 덜 취약한 모습을 보이기도 한다.
* 하이퍼 파라미터를 덜 설정해도 자동적으로 세팅

In [76]:
from sklearn.ensemble import AdaBoostRegressor

start = time.time()

model = AdaBoostRegressor()

model.fit(X_train,Y_train)

time_len = (time.time() - start)

print('정확도 : {}'.format(rmsle(model.predict(X_test), Y_test)))
print('수행시간 : {}'.format(time_len))

정확도 : 0.7338709166803171
수행시간 : 0.7062323093414307


## 앙상블모델 - 그라디언트 부스트
### Ensemble Model - Gradient Boost
* 여러개의 결정트리를 묶어 강력한 모델을 만드는 또 다른 앙상블 기법
* 회귀와 분류에 모두 사용할 수 있음
* 랜덤포레스트와 달리 이진 트리의 오차를 보완하는 방식으로 순차적으로 트리를 만든다.
* 무작위성이 없고 강력한 사전 가지치기가 사용 됨

In [18]:
from sklearn.ensemble import GradientBoostingRegressor

start = time.time()

model = GradientBoostingRegressor()

model.fit(X_train,Y_train)

time_len = (time.time() - start )

print('정확도 : {}'.format(rmsle(model.predict(X_test), Y_test)))
print('수행시간 : {}'.format(time_len))

정확도 : 0.05356791948388584
수행시간 : 0.34702038764953613


## 엑스지 부스트
### XGBoost
* GradientBoost 기반
* 단일 시스템에서 사용한 기존 일반적 솔루션보다 10배 이상 빠르게 실행
* 과적합 방지가 가능한 규제가 포함되어있음
* 조기종료 제공 

In [20]:
from xgboost import XGBRegressor

start = time.time()

model = XGBRegressor()

model.fit(X_train, Y_train)

time_len = (time.time() - start)

print('정확도 : {}'.format(rmsle(model.predict(X_test), Y_test)))
print('수행시간 : {}'.format(time_len))

[06:12:17] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
정확도 : 0.06347195938469834
수행시간 : 0.4625411033630371


## 라이트 지비엠
### LightGBM

* 그래디언트 부스팅 결정트리 프레임워크

* Gradient-based One-Side Sampling(GOSS)과 Exclusive Feature Bundling(EFB)라는 두 가지 기술을 이용하여 보완한 프레임워크.

In [80]:
from lightgbm import LGBMRegressor

start = time.time()

model = LGBMRegressor()

model.fit(X_train, Y_train)

time_len = (time.time() - start)

print('정확도 : {}'.format(rmsle(model.predict(X_test), Y_test)))
print('수행시간 : {}'.format(time_len))

정확도 : 0.02393840209049412
수행시간 : 0.27855730056762695


## 선형 회귀
### Linear Regression

In [81]:
import time
from sklearn.linear_model import LinearRegression #선형 회귀 모델

start = time.time()  # 시작 시간 저장

model = LinearRegression()

model.set_params()

model.fit(X_train, Y_train)

time_len = (time.time() - start)

print('정확도 : {}'.format(rmsle(model.predict(X_test), Y_test)))
print('수행시간 : {}'.format(time_len))

정확도 : 1.4277430645784713e-14
수행시간 : 0.01055145263671875


In [82]:
list(zip(map(lambda x: int(round(x)), model.predict(X_test)), Y_test)) #???

[(10, 10),
 (569, 569),
 (274, 274),
 (168, 168),
 (23, 23),
 (215, 215),
 (338, 338),
 (168, 168),
 (525, 525),
 (288, 288),
 (12, 12),
 (458, 458),
 (3, 3),
 (166, 166),
 (24, 24),
 (163, 163),
 (772, 772),
 (4, 4),
 (65, 65),
 (20, 20),
 (211, 211),
 (441, 441),
 (129, 129),
 (190, 190),
 (74, 74),
 (159, 159),
 (179, 179),
 (28, 28),
 (188, 188),
 (11, 11),
 (327, 327),
 (133, 133),
 (198, 198),
 (813, 813),
 (95, 95),
 (342, 342),
 (56, 56),
 (193, 193),
 (374, 374),
 (10, 10),
 (8, 8),
 (23, 23),
 (116, 116),
 (346, 346),
 (11, 11),
 (647, 647),
 (261, 261),
 (7, 7),
 (2, 2),
 (169, 169),
 (281, 281),
 (236, 236),
 (15, 15),
 (395, 395),
 (297, 297),
 (82, 82),
 (394, 394),
 (236, 236),
 (95, 95),
 (7, 7),
 (13, 13),
 (194, 194),
 (1, 1),
 (364, 364),
 (6, 6),
 (514, 514),
 (26, 26),
 (10, 10),
 (299, 299),
 (400, 400),
 (45, 45),
 (191, 191),
 (89, 89),
 (381, 381),
 (342, 342),
 (301, 301),
 (45, 45),
 (73, 73),
 (2, 2),
 (7, 7),
 (61, 61),
 (209, 209),
 (140, 140),
 (200, 200)